In [1]:
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
import importlib

In [2]:
rec1 = Recording('E:/VISION/129/Recording_Session/121622061907_2023Feb13022044.bag')
rec2 = Recording('E:/VISION/129/Recording_Session/123122061443_2023Feb13022044.bag')
rec1.load_calibration('121622061907_cal.txt')
rec2.load_calibration('123122061443_cal.txt')



array([[-0.76596981, -0.47690493,  0.46312463, -0.43199882],
       [-0.65983385,  0.45945311, -0.61818653,  0.47658443],
       [ 0.0808824 , -0.76817775, -0.65726215,  0.49703872],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [3]:
d = DualRecording(rec1,rec2)



In [4]:
d.start_stream()

In [5]:
d.get_frames()

((<pyrealsense2.frame Z16 #1160 @1676258444174.644043>,
  <pyrealsense2.frame RGB8 #1194 @1676258444174.980957>),
 (<pyrealsense2.frame Z16 #1121 @1676258444195.972412>,
  <pyrealsense2.frame RGB8 #1123 @1676258444196.309326>))

In [13]:
t = Tracker(ref_surface=d.pcd)


[Open3D INFO] Picked point #1151320 (-0.048, 0.031, 0.16) to add in queue.
[Open3D INFO] Picked point #200106 (0.096, -0.079, 0.16) to add in queue.


In [8]:
d.pcd

RuntimeError: [Open3D Error] (void __cdecl open3d::t::geometry::PointCloud::SetPointAttr(const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class open3d::core::Tensor &)) C:\Users\RemoveTheMask\MyDocs\Code\Open3D\cpp\open3d/t/geometry/PointCloud.h:160: Attribute device CPU:0 != Pointcloud's device CUDA:0.


In [14]:
d.compute_pcd(True)

PointCloud on CUDA:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [17]:
a1=0
b1=0
a2=0
b2=0
for i in range(100):
    a1 += time.time()
    p = d.compute_pcd(True)
    b1 += time.time()

    a2 += time.time()
    t.get_motion(p)
    b2 += time.time()
print(b1-a1)
b2-a2

Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258455194.96 1488 1676258455317.0886 1452
Timestamps:1676258457782.2197 1565 1676258457904.2498 1529
Timestamps:1676258458823.7617 1596 1676258458945.8157 1560
26.4205322265625


15.67352294921875

-0.21874094009399414

In [12]:
t._t_guess

[[0.999998 -0.00176567 0.00116396 -0.000151304],
 [0.00176516 0.999998 0.000434744 -0.000104731],
 [-0.00116473 -0.000432688 0.999999 -0.000153702],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CPU:0, 0x1e3f23fa0b0]

In [35]:
pcd_cuda = o3d.t.geometry.PointCloud(o3d.core.Device('CUDA:1'))
pcd_cpu = o3d.t.geometry.PointCloud(o3d.core.Device('CPU:0'))

In [11]:
pcd.cuda()

PointCloud on CUDA:0 [0 points].
Attributes: None.

In [12]:
pcd.cpu()

PointCloud on CPU:0 [0 points].
Attributes: None.

In [13]:
pcd

PointCloud on CUDA:1 [0 points].
Attributes: None.

In [36]:
rec1._rs_pc.map_to(rec1.frame[1])

In [37]:
dat=rec1._rs_pc.calculate(rec1.frame[0])

In [38]:
depth_dat = np.asanyarray(dat.get_vertices()).view(np.float32).reshape([-1, 3])

In [39]:
col_dat = np.asanyarray(rec1.frame[1].get_data()).reshape([-1, 3])

In [40]:
pcd_cuda.point["positions"] = depth_dat
pcd_cpu.point["positions"] = depth_dat

In [41]:
pcd_cuda.point["colors"] = col_dat
pcd_cpu.point["positions"] = depth_dat

In [58]:
pcd_cpu.cuda().device

CUDA:0

In [53]:
pcd_cpu.device

CPU:0

In [56]:
pcd_cuda.device

CUDA:1

In [57]:
pcd_cuda.cpu().device

CPU:0

In [1]:
1000%100


0

In [3]:
import numpy as np
d = [1,2,3]
np.savetxt('test.txt',d)

In [6]:
s = d.pcd.cuda()

RuntimeError: [Open3D Error] (class std::shared_ptr<class open3d::core::MemoryManagerDevice> __cdecl open3d::core::MemoryManager::GetMemoryManagerDevice(const class open3d::core::Device &)) D:\a\Open3D\Open3D\cpp\open3d\core\MemoryManager.cpp:123: Unsupported device "CUDA:0". Set BUILD_CUDA_MODULE=ON to compile for CUDA support and BUILD_SYCL_MODULE=ON to compile for SYCL support.


In [23]:
d.pcd.cpu()

PointCloud on CPU:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [50]:
o3d.cuda

<module 'open3d.cuda' (<_frozen_importlib_external._NamespaceLoader object at 0x000001C077375D20>)>

In [63]:
hasattr(o3d,'cuda')

True

In [68]:
DEVICE = o3d.core.Device('cuda:0')

In [72]:
o3d.t.geometry.PointCloud(DEVICE).to_legacy()

PointCloud with 0 points.

In [75]:
o3d.core.Tensor(np.asarray([1,2]),device=DEVICE)

[1 2]
Tensor[shape={2}, stride={1}, Int32, CUDA:0, 0x13030d7a00]

In [15]:
Tracker

spygrt.tracker.Tracker

In [6]:
d.compute_pcd(True)

RuntimeError: [Open3D Error] (void __cdecl open3d::t::geometry::PointCloud::SetPointAttr(const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class open3d::core::Tensor &)) C:\Users\RemoveTheMask\MyDocs\Code\Open3D\cpp\open3d/t/geometry/PointCloud.h:160: Attribute device CPU:0 != Pointcloud's device CUDA:0.


In [20]:
d._pose

(array([[-0.72437882,  0.46467108, -0.50862086,  0.37932215],
        [ 0.68838706,  0.51730415, -0.50779807,  0.43379043],
        [ 0.02716157, -0.71820357, -0.69482736,  0.50452895],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[-0.76596981, -0.47690493,  0.46312463, -0.43199882],
        [-0.65983385,  0.45945311, -0.61818653,  0.47658443],
        [ 0.0808824 , -0.76817775, -0.65726215,  0.49703872],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [9]:
rec1.pcd + rec2.pcd

RuntimeError: [Open3D Error] (void __cdecl open3d::t::geometry::PointCloud::SetPointAttr(const class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > &,const class open3d::core::Tensor &)) C:\Users\RemoveTheMask\MyDocs\Code\Open3D\cpp\open3d/t/geometry/PointCloud.h:160: Attribute device CPU:0 != Pointcloud's device CUDA:0.


In [88]:
f= rec1.frame
dev = o3d.core.Device('cuda:0')
rec1._pcd = o3d.t.geometry.PointCloud(device=dev)
rec2._pcd = o3d.t.geometry.PointCloud(device=dev)
a = time.time()
for rec in [rec1,rec2]:
    rec._rs_pc.map_to(rec.frame[1])
    # Get the raw point cloud data
    dat = rec._rs_pc.calculate(rec.frame[0])
    # Changes how the data is "viewed" without making changes to the memory. Results in a list of 3D coordinates
    depth_dat = np.asanyarray(dat.get_vertices()).view(np.float32).reshape([-1, 3])
    # Reformat the colour data into a list of RGB values
    col_dat = np.asanyarray(rec.frame[1].get_data()).reshape([-1, 3])
    col_cuda = o3d.core.Tensor(col_dat,device=dev)
    rec._pcd.point["colors"] = col_cuda
    depth_cuda = o3d.core.Tensor(depth_dat,device=dev)
    rec._pcd.point["positions"] = depth_cuda
    rec._pcd.transform(o3d.core.Tensor(rec._pose, device=dev))
rec1.pcd + rec2.pcd
b= time.time()
a-b

-0.0476689338684082

In [85]:
dev = o3d.core.Device('cpu:0')
rec1._pcd = o3d.t.geometry.PointCloud(device=dev)
rec2._pcd = o3d.t.geometry.PointCloud(device=dev)
a = time.time()
for rec in [rec1,rec2]:
    rec._rs_pc.map_to(rec.frame[1])
    # Get the raw point cloud data
    dat = rec._rs_pc.calculate(rec.frame[0])
    # Changes how the data is "viewed" without making changes to the memory. Results in a list of 3D coordinates
    depth_dat = np.asanyarray(dat.get_vertices()).view(np.float32).reshape([-1, 3])
    # Reformat the colour data into a list of RGB values
    col_dat = np.asanyarray(rec.frame[1].get_data()).reshape([-1, 3])
    col_cuda = o3d.core.Tensor(col_dat,device=dev)
    rec._pcd.point["colors"] = col_cuda
    depth_cuda = o3d.core.Tensor(depth_dat,device=dev)
    rec._pcd.point["positions"] = depth_cuda
    rec._pcd.transform(o3d.core.Tensor(rec._pose, device=dev))
rec1._pcd = rec1._pcd.cuda()
rec2._pcd = rec2._pcd.cuda()
rec1.pcd + rec2.pcd
b= time.time()
a-b

-0.058202505111694336

In [80]:
rec1.pcd

PointCloud on CPU:0 [921600 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {921600, 3}).

In [ ]:
a = time.time()
t.get_motion(p)
b= time.time()